
# NaviGator Toolkit API Model Eval Demo

This notebook is designed to test connection to the NaviGator Toolkit API ([see here for more information](https://it.ufl.edu/ai/navigator-toolkit/)). You will need a NaviGator API key. The key should be stored in a `.json` file with the following format:

    {
      "OPENAI_API_KEY" : "Put your key here in the quotes",
      "base_url" : "https://api.ai.it.ufl.edu/"
    }

We suggest putting that in your home directory, to minimize the chances of accidentally adding and committing the file to a git repo. Remember that anyone with your API key can use NaviGator as you! 

In [1]:
import openai
import os
import json

## Load `.json` file with your key and api endpoint URL

In [3]:
# Set the path to your jsnkey file
key_file = '/home/magitz/navigator_api_keys.json'


# Load the JSON file
with open(key_file, 'r') as file:
    data = json.load(file)

# Extract the values
OPENAI_API_KEY = data.get('OPENAI_API_KEY')
base_url = data.get('base_url')

# Set the environment variable
os.environ['TOOLKIT_API_KEY'] = OPENAI_API_KEY

## Test connectivity and get model list

Reply should list the models that are available with your API key. An example, truncated, output is:

    SyncPage[Model](data=[Model(id='llama-3.1-70b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='sfr-embedding-mistral', created=1677610602, object='model', owned_by='openai'),...)], object='list')

In [33]:
# Check list of available models
client = openai.OpenAI(
    api_key=os.environ.get("TOOLKIT_API_KEY"),
    base_url=base_url
)

response = client.models.list()
 
print(response)

SyncPage[Model](data=[Model(id='codestral-22b', created=1677610602, object='model', owned_by='openai'), Model(id='whisper-large-v3', created=1677610602, object='model', owned_by='openai'), Model(id='mistral-7b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='mixtral-8x7b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='gte-large-en-v1.5', created=1677610602, object='model', owned_by='openai'), Model(id='llama-3.1-8b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='llama-3.1-70b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='flux.1-dev', created=1677610602, object='model', owned_by='openai'), Model(id='flux.1-schnell', created=1677610602, object='model', owned_by='openai'), Model(id='sfr-embedding-mistral', created=1677610602, object='model', owned_by='openai'), Model(id='nomic-embed-text-v1.5', created=1677610602, object='model', owned_by='openai')], object='list')


## Parse list of available models, keep only the models you want to test

In [34]:

# Not all models in NaviGator are LLMs
models_to_test=['codestral-22b',
 'mistral-7b-instruct',
 'mixtral-8x7b-instruct',
 'llama-3.1-8b-instruct',
 'llama-3.1-70b-instruct']


print("Making a list of available models:")
model_list=[]
for model in response.data:
    print(f'Model name: {model.id}')
    if model.id in models_to_test:
        model_list.append(model.id)
    
print(f'Your API key has access to {len(response.data)} models, of those {len(model_list)} are in your list of {len(models_to_test)} models to test.')

Making a list of available models:
Model name: codestral-22b
Model name: whisper-large-v3
Model name: mistral-7b-instruct
Model name: mixtral-8x7b-instruct
Model name: gte-large-en-v1.5
Model name: llama-3.1-8b-instruct
Model name: llama-3.1-70b-instruct
Model name: flux.1-dev
Model name: flux.1-schnell
Model name: sfr-embedding-mistral
Model name: nomic-embed-text-v1.5
Your API key has access to 11 models, of those 5 are in your list of 5 models to test.


In [35]:
model_list

['codestral-22b',
 'mistral-7b-instruct',
 'mixtral-8x7b-instruct',
 'llama-3.1-8b-instruct',
 'llama-3.1-70b-instruct']

## Create sample query to pass to all models

In [36]:
query = '''Please write a Python function that reads in data from the file demo.csv. 
           The file has the following columns: Age, Test_score, Hours_studied, Course_load,
           and 1/0 if a student passes the final exam. Please make a Logistic model to predict
           is a student passes the exam based on the other data. Please use 5-fold cross-validation.
           Plot a confusion matrix, and other measures to evaluate the performance of the model. 
           Comment all code with explanations and possible parameter adjustment to improve the model.'''

## Send the query to the models and save the results

In [37]:
out_file_base = 'code_' # prefix for the output file, will have model name added

print(f'Sending the following query to each model in the list: \n {query}')

for model in model_list:
    print(f'Sending query to {model}')
    response = client.chat.completions.create(
            model=model,
            messages = [
                    {
                        "role": "user",
                        "content": query
                    }
                ]
            )
    # Write reponse to file    
    try:
        out_file = out_file_base + model + '.md'
        with open(out_file, 'w') as f:
            f.write(response.choices[0].message.content)
        f.close()
    except Exception as e:
        print('Error writing file.', e)
print('Queries finished.')

Sending the following query to each model in the list: 
 Please write a Python function that reads in data from the file demo.csv. 
           The file has the following columns: Age, Test_score, Hours_studied, Course_load,
           and 1/0 if a student passes the final exam. Please make a Logistic model to predict
           is a student passes the exam based on the other data. Please use 5-fold cross-validation.
           Plot a confusion matrix, and other measures to evaluate the performance of the model. 
           Comment all code with explanations and possible parameter adjustment to improve the model.
Sending query to codestral-22b
Sending query to mistral-7b-instruct
Sending query to mixtral-8x7b-instruct
Sending query to llama-3.1-8b-instruct
Sending query to llama-3.1-70b-instruct
Queries finished.
